Import

In [1]:
!git clone https://github.com/AndreasHammerKU/CardiacCTAnalysis.git
%cd CardiacCTAnalysis
!git checkout baseline_implementation

Cloning into 'CardiacCTAnalysis'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 162 (delta 75), reused 111 (delta 33), pack-reused 0 (from 0)
Receiving objects: 100% (162/162), 788.94 KiB | 5.93 MiB/s, done.
Resolving deltas: 100% (75/75), done.
/content/CardiacCTAnalysis
Branch 'baseline_implementation' set up to track remote branch 'baseline_implementation' from 'origin'.
Switched to a new branch 'baseline_implementation'


Imports from Github Repository

In [2]:
!pip install dash
import numpy as np

# Custom Imports
import utils.io_utils as io
import utils.logger as logs
from baseline.BaseEnvironment import MedicalImageEnvironment
from utils.io_utils import DataLoader
from baseline.BaseAgent import DQNAgent

# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

dataset_folder = '/content/drive/MyDrive/Data'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0
Mounted at /content/drive


Hyperparameters

In [3]:
MAX_STEPS = 1000
EPISODES = 500
IMAGE_INTERVAL = 2
AGENTS = 1
N_SAMPLE_POINTS = 5

In [4]:
debug = False

# Colab not enough RAM
preload_images = False
logger = logs.setup_logger(debug)

dataLoader = DataLoader(dataset_folder)

Training

In [5]:
# Initialize training environment
env = MedicalImageEnvironment(logger=logger,
                              dataLoader=dataLoader,
                              image_list=['n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', 'n15', 'n16', 'n17', 'n18', 'n19', 'n20', 'n21', 'n22', 'n23', 'n24', 'n25', 'n26', 'n27', 'n28', 'n29', 'n30'],
                              agents=AGENTS,
                              n_sample_points=N_SAMPLE_POINTS,
                              preload_images=preload_images)
agent = DQNAgent(environment=env,
                 task="train",
                 logger=logger,
                 state_dim=env.state_size,
                 action_dim=env.n_actions,
                 agents=AGENTS,
                 max_steps=MAX_STEPS,
                 episodes=EPISODES,
                 image_interval=IMAGE_INTERVAL)

agent.train_dqn()

INFO:Logger:Episode 1: Total Reward = tensor([-3.5829], device='cuda:0', dtype=torch.float64) Reached Goal False Closest Point = 2.8284271247461903 Furthest Point = 39.63584236521283
INFO:Logger:Episode 2: Total Reward = tensor([-59.8061], device='cuda:0', dtype=torch.float64) Reached Goal False Closest Point = 3.1622776601683795 Furthest Point = 75.45859791965393
INFO:Logger:Episode 3: Total Reward = tensor([2.4544], device='cuda:0', dtype=torch.float64) Reached Goal False Closest Point = 4.58257569495584 Furthest Point = 77.08436936240706
INFO:Logger:Episode 4: Total Reward = tensor([9.6394], device='cuda:0', dtype=torch.float64) Reached Goal False Closest Point = 3.0 Furthest Point = 49.4368283772331
INFO:Logger:Episode 5: Total Reward = tensor([-18.7278], device='cuda:0', dtype=torch.float64) Reached Goal False Closest Point = 5.0 Furthest Point = 41.10960958218893
INFO:Logger:Episode 6: Total Reward = tensor([8.1692], device='cuda:0', dtype=torch.float64) Reached Goal False Closes

Evaluate

In [8]:
logger = logs.setup_logger(True)
dataLoader = DataLoader(dataset_folder)

env = MedicalImageEnvironment(logger=logger,
                              task="eval",
                              dataLoader=dataLoader,
                              image_list=['n31', 'n32', 'n33', 'n34', 'n35', 'n36', 'n37', 'n38', 'n39', 'n40'],
                              agents=AGENTS,
                              n_sample_points=N_SAMPLE_POINTS)
agent = DQNAgent(environment=env,
                 task="eval",
                 logger=logger,
                 state_dim=env.state_size,
                 action_dim=env.n_actions,
                 agents=AGENTS,
                 model_path="latest-model.pt",
                 max_steps=100,
                 episodes=10
                 )

agent.evaluate_dqn()

/content/CardiacCTAnalysis/baseline/BaseAgent.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.policy_net.load_state_dict(torch.load(model_path, map_location=self.d

Evaluation Episode 1: Total Reward = tensor([-53.7328], device='cuda:0', dtype=torch.float64) | Closest Point: 6.324555320336759 | Furthest Point: 76.02631123499285


DEBUG:Logger:Loaded image: n32 with ground truth [256. 224. 197.] and starting point [234. 225. 206.]


Evaluation Episode 2: Total Reward = tensor([-74.7029], device='cuda:0', dtype=torch.float64) | Closest Point: 24.186773244895647 | Furthest Point: 98.49365461794989


DEBUG:Logger:Loaded image: n33 with ground truth [206. 183. 272.] and starting point [182. 186. 278.]


...

Evaluation Episode 8: Total Reward = tensor([-25.6311], device='cuda:0', dtype=torch.float64) | Closest Point: 10.0 | Furthest Point: 54.85435260760991


DEBUG:Logger:Loaded image: n39 with ground truth [227. 212. 358.] and starting point [206. 213. 351.]


Evaluation Episode 9: Total Reward = tensor([-65.4086], device='cuda:0', dtype=torch.float64) | Closest Point: 22.22611077089287 | Furthest Point: 87.56711711595854


DEBUG:Logger:Loaded image: n40 with ground truth [233. 234. 238.] and starting point [217. 233. 238.]


Evaluation Episode 10: Total Reward = tensor([-53.0340], device='cuda:0', dtype=torch.float64) | Closest Point: 16.06237840420901 | Furthest Point: 69.06518659932803

===== Evaluation Summary =====
Average Reward: -43.19
Success Rate: 0.00%
Average Closest Distance: 17.49
Average Furthest Distance: 67.23
